# Tugas Kolaborasi Memprediksi Saham Perusahaan Bukalapak

1. Dio Adya Wicaksono - 200411100066 (Ketua)
2. Dimas Mahardika - 200411100093 (Anggota)

Link Github : https://github.com/errjak/uaspsd

## Read Data

In [ ]:
import pandas as pd
data = pd.read_csv(
    'https://raw.githubusercontent.com/errjak/dataset/main/BUKA.JK.csv')

data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2022-06-15,272.0,278.0,266.0,278.0,278.0,275652400
1,2022-06-16,284.0,290.0,274.0,278.0,278.0,372853400
2,2022-06-17,274.0,284.0,272.0,284.0,284.0,290985700
3,2022-06-20,280.0,286.0,274.0,284.0,284.0,172441200
4,2022-06-21,286.0,300.0,282.0,298.0,298.0,423336400
...,...,...,...,...,...,...,...
242,2023-06-08,214.0,220.0,208.0,212.0,212.0,209435400
243,2023-06-09,216.0,220.0,212.0,216.0,216.0,235016600
244,2023-06-12,216.0,224.0,214.0,216.0,216.0,223424400
245,2023-06-13,216.0,224.0,206.0,210.0,210.0,256715100


In [ ]:
# ambil kolom date dan close saja untuk memprediksi harga saham
data = data[['Date', 'Close']]
data

,Date,Close
0,2022-06-15,278.0
1,2022-06-16,278.0
2,2022-06-17,284.0
3,2022-06-20,284.0
4,2022-06-21,298.0
...,...,...
242,2023-06-08,212.0
243,2023-06-09,216.0
244,2023-06-12,216.0
245,2023-06-13,210.0


In [ ]:
data.shape

(247, 2)

In [ ]:
from array import array
import numpy as np

def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [ ]:
# define input sequence from bukalapak.csv
df = data
series = df['Close']
# choose a number of time steps
n_steps = 3
# split into samples
X, y = split_sequence(series, n_steps)

# show each sample
for i in range(len(X)):
    print(X[i], y[i])


[278. 278. 284.] 284.0
[278. 284. 284.] 298.0
[284. 284. 298.] 286.0
[284. 298. 286.] 280.0
[298. 286. 280.] 288.0
[286. 280. 288.] 284.0
[280. 288. 284.] 272.0
[288. 284. 272.] 264.0
[284. 272. 264.] 282.0
[272. 264. 282.] 266.0
[264. 282. 266.] 256.0
[282. 266. 256.] 256.0
[266. 256. 256.] 252.0
[256. 256. 252.] 254.0
[256. 252. 254.] 258.0
[252. 254. 258.] 256.0
[254. 258. 256.] 256.0
[258. 256. 256.] 252.0
[256. 256. 252.] 252.0
[256. 252. 252.] 254.0
[252. 252. 254.] 256.0
[252. 254. 256.] 268.0
[254. 256. 268.] 288.0
[256. 268. 288.] 280.0
[268. 288. 280.] 296.0
[288. 280. 296.] 282.0
[280. 296. 282.] 280.0
[296. 282. 280.] 296.0
[282. 280. 296.] 300.0
[280. 296. 300.] 294.0
[296. 300. 294.] 292.0
[300. 294. 292.] 298.0
[294. 292. 298.] 322.0
[292. 298. 322.] 332.0
[298. 322. 332.] 318.0
[322. 332. 318.] 310.0
[332. 318. 310.] 304.0
[318. 310. 304.] 308.0
[310. 304. 308.] 320.0
[304. 308. 320.] 330.0
[308. 320. 330.] 334.0
[320. 330. 334.] 332.0
[330. 334. 332.] 332.0
[334. 332. 

## Preprocessing Data With MinMaxScaler

In [ ]:
# prepoceesing data dengan MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
X = scaler.fit_transform(X)

X

array([[0.59124088, 0.59124088, 0.6350365 ],
       [0.59124088, 0.6350365 , 0.6350365 ],
       [0.6350365 , 0.6350365 , 0.73722628],
       [0.6350365 , 0.73722628, 0.64963504],
       [0.73722628, 0.64963504, 0.60583942],
       [0.64963504, 0.60583942, 0.66423358],
       [0.60583942, 0.66423358, 0.6350365 ],
       [0.66423358, 0.6350365 , 0.54744526],
       [0.6350365 , 0.54744526, 0.48905109],
       [0.54744526, 0.48905109, 0.62043796],
       [0.48905109, 0.62043796, 0.50364964],
       [0.62043796, 0.50364964, 0.43065693],
       [0.50364964, 0.43065693, 0.43065693],
       [0.43065693, 0.43065693, 0.40145985],
       [0.43065693, 0.40145985, 0.41605839],
       [0.40145985, 0.41605839, 0.44525547],
       [0.41605839, 0.44525547, 0.43065693],
       [0.44525547, 0.43065693, 0.43065693],
       [0.43065693, 0.43065693, 0.40145985],
       [0.43065693, 0.40145985, 0.40145985],
       [0.40145985, 0.40145985, 0.41605839],
       [0.40145985, 0.41605839, 0.43065693],
       [0.

## Reduksi Dimensi dengan PCA

In [ ]:
# reduksi dimensi dengan PCA
from sklearn.decomposition import PCA
pca = PCA(n_components=1)
X = pca.fit_transform(X)

X

array([[-0.14820066],
       [-0.17365776],
       [-0.25790507],
       [-0.26652927],
       [-0.24858149],
       [-0.2069569 ],
       [-0.19896549],
       [-0.16456368],
       [-0.06312635],
       [-0.05532915],
       [-0.03065421],
       [ 0.00451755],
       [ 0.11363667],
       [ 0.17224354],
       [ 0.18075232],
       [ 0.17201404],
       [ 0.15516893],
       [ 0.14698198],
       [ 0.17224354],
       [ 0.18921494],
       [ 0.19742498],
       [ 0.18047666],
       [ 0.11287892],
       [-0.0309978 ],
       [-0.13202229],
       [-0.2491437 ],
       [-0.22444567],
       [-0.22327376],
       [-0.22413472],
       [-0.30060924],
       [-0.3588834 ],
       [-0.34163633],
       [-0.33352951],
       [-0.45220171],
       [-0.62135219],
       [-0.70457828],
       [-0.65300954],
       [-0.53532459],
       [-0.49344742],
       [-0.53618555],
       [-0.6460855 ],
       [-0.75545721],
       [-0.80564762],
       [-0.81383457],
       [-0.81396087],
       [-0

In [ ]:
# split data train dan test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, shuffle=False, test_size=0.2)

# tampilkan shape train dan test
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(195, 1) (195,)
(49, 1) (49,)


## Model Naive Bayess

In [ ]:
# buat model naive bayes
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

# train model
model.fit(X_train, y_train)

# prediksi data test
y_NB = model.predict(X_test)

# MAPE
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, y_NB)
print('MAPE Naive Bayes : %.3f' % mape)

MAPE Naive Bayes : 0.245


## Model Decision Tree

In [ ]:
# buat model decision tree
from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()

# train model
model.fit(X_train, y_train)

# prediksi data test
y_DT = model.predict(X_test)


# MAPE
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, y_DT)
print('MAPE Decision Tree : %.3f' % mape)

MAPE Decision Tree : 0.062


## Model MLP

In [ ]:
# buat model MLP
from sklearn.neural_network import MLPRegressor
from sklearn.exceptions import ConvergenceWarning
import warnings

max_iter = 500
tolerance = 0.0001

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning,module="sklearn")
    model = MLPRegressor(hidden_layer_sizes=(100, 100),max_iter=max_iter,tol=tolerance)

    # train model
    model.fit(X_train, y_train)

    # prediksi data test
    y_MLP = model.predict(X_test)

# MAPE
from sklearn.metrics import mean_absolute_percentage_error
mape = mean_absolute_percentage_error(y_test, y_MLP)
print('MAPE MLP : %.3f' % mape)

MAPE MLP : 0.122
